In [1]:
import pandas as pd
df_tweets = pd.read_csv("data/tweets_elections_raw.csv",
                        sep = ";", 
                        encoding = "utf-8",
                        quotechar = "\"",
                        dtype={'id': object, 'user_id': object})  


# Cluster 1: Boric, CLuster 2: Kast
df_tweets["source"] = "twitter"

# Only rows with language = "es"
df_tweets = df_tweets[df_tweets["language"] == "es"]

df_tweets = df_tweets.rename(columns={'tweet': 'text', 'cluster': 'candidato'})
df_tweets = df_tweets[["user_id", "date", "text", "candidato", "source"]]
df_tweets = df_tweets.reset_index(drop=True)

# Replace candidato == 1 with "Boric" and candidato == 2 with "Kast" 
df_tweets["candidato"] = df_tweets["candidato"].replace(1, "Boric")
df_tweets["candidato"] = df_tweets["candidato"].replace(2, "Kast")

df_tweets.head(3)


,user_id,date,text,candidato,source
0,1442114017238028298,2021-11-03 00:00:02,@joseantoniokast @DanyJaneLpez1 Aquí lo que fr...,Kast,twitter
1,1442114017238028298,2021-11-03 00:03:01,@joseantoniokast @DanyJaneLpez1 Tenemos que un...,Kast,twitter
2,1442114017238028298,2021-11-03 00:06:37,@joseantoniokast @DanyJaneLpez1 Somos libres s...,Kast,twitter


In [2]:
df_wsp = pd.read_pickle("data/wsp_elections_raw.pkl")

df_wsp = df_wsp.rename(columns={'remote_resource': 'user_id', 'data': 'text', 'key_remote_jid': 'group_id'})
df_wsp = df_wsp[["user_id", "date", "text", "candidato", "key_from_me"]]

# Remove rows with key_from_me = 1
df_wsp = df_wsp[df_wsp.key_from_me == 0]

df_wsp = df_wsp.dropna(subset=["text"])
# Remove rows with empty text or NaNs:
df_wsp = df_wsp[df_wsp.text.notnull()]
df_wsp = df_wsp[df_wsp.text != '']
del df_wsp["key_from_me"]
df_wsp = df_wsp.reset_index(drop=True)
df_wsp["source"] = "whatsapp"

df_wsp.head(3)


,user_id,date,text,candidato,source
0,56973794459@s.whatsapp.net,2021-11-03 21:34:46,*AtreviDos*: Resumen Diario ✌🏼\n\n1️⃣ Por nues...,Kast,whatsapp
1,56973794459@s.whatsapp.net,2021-11-04 20:45:55,🔴 *AHORA*\n\nExisten prejuicios y caricaturas ...,Kast,whatsapp
2,56973794459@s.whatsapp.net,2021-11-04 21:45:32,*AtreviDos*: Resumen Diario ✌🏼\n\n1️⃣ *Atrévet...,Kast,whatsapp


In [3]:
df = pd.concat([df_tweets, df_wsp], ignore_index=True)

In [4]:
from preprocess import SpanishPreprocess 
import swifter 

sp = SpanishPreprocess(lower=True, remove_url=True, remove_hashtags = True, convert_emoticons=False, convert_emojis=False, normalize_inclusive_language=False, reduce_spam=True,
                        remove_vowels_accents = True, remove_punctuation=True, remove_unprintable=True, remove_numbers=True, remove_stopwords=False, stopwords_list=None, stem=False)


In [5]:
df["text"] = df["text"].swifter.apply(sp.transform)

Pandas Apply:   0%|          | 0/1190797 [00:00<?, ?it/s]

In [6]:
df = df[df.text.notnull()]
df = df[df.text != '']
df = df[df["text"].apply(lambda x: type(x) == str)]
df = df.reset_index(drop=True)

In [7]:
from classifiers import SpanishClassifier
sc = SpanishClassifier(model_name="sexist_analysis")
t1 = "Las mujeres son tontas"
p = sc.predict(t1)

: 

: 

In [ ]:
df["sexist_analysis"] = df["text"].swifter.apply(lambda x: sc.predict(x))


IndexError: index out of range in self

In [ ]:
for index, row in df.iterrows():
    try:
        sc.predict(row["text"])
    except:
        print(row["text"])
        sc.predict(x)
        

NameError: name 'df' is not defined